In [2]:
import pandas as pd
import re
from IPython.display import display, HTML

def contiene_ejemplo(texto):
    if pd.isna(texto): return False
    return bool(re.search(r'por ejemplo|ej\.|como es el caso|tal como|caso de', texto, re.IGNORECASE))

def contiene_datos(texto):
    if pd.isna(texto): return False
    return bool(re.search(r'\d{4}|\d+%|millones|miles|ha\b|tn\b', texto, re.IGNORECASE))

def usa_lenguaje_causal(texto):
    if pd.isna(texto): return False
    return bool(re.search(r'debido a|gracias a|por lo tanto|consecuencia|porque|ya que|dado que', texto, re.IGNORECASE))

def tiene_vision_futuro(texto):
    if pd.isna(texto): return False
    return bool(re.search(r'futuro|largo plazo|tendencia|será|permitirá|impactará|podría|debería', texto, re.IGNORECASE))

def crear_scorecard_enriquecido(dataframe):
    participacion_df = dataframe.groupby(['email', 'Bloque temático']).agg(
        respuestas_por_bloque=('pk_id', 'count'),
        argumentos_por_bloque=('reason', 'count')
    ).reset_index()

    argumentos_df = dataframe.dropna(subset=['reason']).copy()
    
    # --- Lógica de scoring movida DENTRO de la función ---
    argumentos_df['contiene_ejemplo'] = argumentos_df['reason'].apply(contiene_ejemplo)
    argumentos_df['contiene_datos'] = argumentos_df['reason'].apply(contiene_datos)
    argumentos_df['usa_lenguaje_causal'] = argumentos_df['reason'].apply(usa_lenguaje_causal)
    argumentos_df['tiene_vision_futuro'] = argumentos_df['reason'].apply(tiene_vision_futuro)
    argumentos_df['score_calidad'] = argumentos_df[['contiene_ejemplo', 'contiene_datos', 'usa_lenguaje_causal', 'tiene_vision_futuro']].sum(axis=1)
    # --- Fin de la lógica movida ---

    calidad_agregada_df = argumentos_df.groupby(['email', 'Bloque temático']).agg(
        score_calidad_promedio=('score_calidad', 'mean'),
        score_calidad_total=('score_calidad', 'sum')
    ).round(2).reset_index()

    scorecard_enriquecido = pd.merge(participacion_df, calidad_agregada_df, on=['email', 'Bloque temático'], how='left')
    scorecard_enriquecido.fillna(0, inplace=True)
    return scorecard_enriquecido

# --- Carga de Datos y Ejecución ---
nombre_archivo_excel = 'tablabase_plana3.xlsx'
try:
    df = pd.read_excel(nombre_archivo_excel)
    print(f"Archivo '{nombre_archivo_excel}' cargado exitosamente.")
    
    scorecard_granular = crear_scorecard_enriquecido(df)
    
    scorecard_general = scorecard_granular.groupby('email').agg(
        total_argumentos_globales=('argumentos_por_bloque', 'sum'),
        score_calidad_promedio_global=('score_calidad_promedio', 'mean')
    ).round(2).reset_index()

    # --- Clasificación por Arquetipos ---
    mediana_cantidad = scorecard_general['total_argumentos_globales'].median()
    mediana_calidad = scorecard_general[scorecard_general['score_calidad_promedio_global'] > 0]['score_calidad_promedio_global'].median()

    print(f"\nPunto de corte (Mediana) - Cantidad de Argumentos: {mediana_cantidad}")
    print(f"Punto de corte (Mediana) - Calidad Promedio de Argumentos: {mediana_calidad}\n")

    def asignar_arquetipo(row):
        # Asegurarse que el score promedio no sea cero para ser considerado de 'alta calidad'
        alta_calidad = row['score_calidad_promedio_global'] >= mediana_calidad and row['score_calidad_promedio_global'] > 0
        alta_cantidad = row['total_argumentos_globales'] >= mediana_cantidad and row['total_argumentos_globales'] > 0
        
        if row['total_argumentos_globales'] == 0:
            return 'Votante'
            
        if alta_calidad and alta_cantidad:
            return 'Experto Ancla'
        elif alta_calidad and not alta_cantidad:
            return 'Especialista Preciso'
        elif not alta_calidad and alta_cantidad:
            return 'Participante Amplio'
        else:
            return 'Votante'

    scorecard_general['arquetipo'] = scorecard_general.apply(asignar_arquetipo, axis=1)

    # --- Mostrar Resultados del Análisis de Arquetipos ---
    print("--- Conteo de Expertos por Arquetipo ---")
    print(scorecard_general['arquetipo'].value_counts().to_string())
    
    print("\n\n--- Ejemplos de Participantes por Arquetipo ---")
    
    columnas_display = ['email', 'total_argumentos_globales', 'score_calidad_promedio_global', 'arquetipo']

    for arquetipo in ['Experto Ancla', 'Especialista Preciso', 'Participante Amplio', 'Votante']:
        header_html = f"<h3>{arquetipo} (ejemplos)</h3>"
        display(HTML(header_html))
        
        ejemplos_df = scorecard_general[scorecard_general['arquetipo'] == arquetipo].sort_values(
            by=['total_argumentos_globales', 'score_calidad_promedio_global'], 
            ascending=False
        )
        
        html_output = ejemplos_df[columnas_display].head(5).to_html(index=False, classes='table table-striped table-bordered text-center', justify='center')
        display(HTML(html_output))

except FileNotFoundError:
    print(f"Error Crítico: No se encontró el archivo '{nombre_archivo_excel}'.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Archivo 'tablabase_plana3.xlsx' cargado exitosamente.

Punto de corte (Mediana) - Cantidad de Argumentos: 0.0
Punto de corte (Mediana) - Calidad Promedio de Argumentos: 0.5

--- Conteo de Expertos por Arquetipo ---
arquetipo
Votante                258
Participante Amplio    110
Experto Ancla           55


--- Ejemplos de Participantes por Arquetipo ---


email,total_argumentos_globales,score_calidad_promedio_global,arquetipo
emilianojk@gmail.com,33,0.70,Experto Ancla
leonca45@gmail.com,29,0.52,Experto Ancla
lgomez@rafaela.inta.gov.ar,20,0.55,Experto Ancla
spicardi@criba.edu.ar,16,1.22,Experto Ancla
rgcarri@mincyt.gov.ar,16,0.88,Experto Ancla


email,total_argumentos_globales,score_calidad_promedio_global,arquetipo


email,total_argumentos_globales,score_calidad_promedio_global,arquetipo
ricardo@qinnova.com.ar,71,0.19,Participante Amplio
dhiglesi@gmail.com,49,0.06,Participante Amplio
adriana.bocco@gmail.com,43,0.44,Participante Amplio
speranza@inti.gob.ar,41,0.24,Participante Amplio
santiagodelsolar@estanciaslauquen.com.ar,36,0.21,Participante Amplio


email,total_argumentos_globales,score_calidad_promedio_global,arquetipo
J_Antonio.Torres@OregonState.edu,0,0.0,Votante
Rodolfo.Bernabeu@uclm.es,0,0.0,Votante
acmatias@correo.inta.gov.ar,0,0.0,Votante
afarroni@pergamino.inta.gov.ar,0,0.0,Votante
alarcon@iquir-conicet.gov.ar,0,0.0,Votante


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import re

# --- 1. FUNCIONES DE ANÁLISIS Y GRÁFICOS ---

# (Incluimos todas las funciones que hemos desarrollado)

def contiene_ejemplo(texto):
    if pd.isna(texto): return False
    return bool(re.search(r'por ejemplo|ej\.|como es el caso|tal como|caso de', texto, re.IGNORECASE))

def contiene_datos(texto):
    if pd.isna(texto): return False
    return bool(re.search(r'\d{4}|\d+%|millones|miles|ha\b|tn\b', texto, re.IGNORECASE))

def usa_lenguaje_causal(texto):
    if pd.isna(texto): return False
    return bool(re.search(r'debido a|gracias a|por lo tanto|consecuencia|porque|ya que|dado que', texto, re.IGNORECASE))

def tiene_vision_futuro(texto):
    if pd.isna(texto): return False
    return bool(re.search(r'futuro|largo plazo|tendencia|será|permitirá|impactará|podría|debería', texto, re.IGNORECASE))

def crear_scorecard_enriquecido(dataframe):
    participacion_df = dataframe.groupby(['email', 'Bloque temático']).agg(
        respuestas_por_bloque=('pk_id', 'count'),
        argumentos_por_bloque=('reason', 'count')
    ).reset_index()

    argumentos_df = dataframe.dropna(subset=['reason']).copy()
    argumentos_df['contiene_ejemplo'] = argumentos_df['reason'].apply(contiene_ejemplo)
    argumentos_df['contiene_datos'] = argumentos_df['reason'].apply(contiene_datos)
    argumentos_df['usa_lenguaje_causal'] = argumentos_df['reason'].apply(usa_lenguaje_causal)
    argumentos_df['tiene_vision_futuro'] = argumentos_df['reason'].apply(tiene_vision_futuro)
    argumentos_df['score_calidad'] = argumentos_df[['contiene_ejemplo', 'contiene_datos', 'usa_lenguaje_causal', 'tiene_vision_futuro']].sum(axis=1)

    calidad_agregada_df = argumentos_df.groupby(['email', 'Bloque temático']).agg(
        score_calidad_promedio=('score_calidad', 'mean'),
        score_calidad_total=('score_calidad', 'sum'),
    ).round(2).reset_index()

    scorecard_enriquecido = pd.merge(participacion_df, calidad_agregada_df, on=['email', 'Bloque temático'], how='left')
    scorecard_enriquecido.fillna(0, inplace=True)
    return scorecard_enriquecido

def generar_grafico_percepciones(dataframe, nombre_categoria):
    try:
        category_df = dataframe[dataframe['Bloque temático'] == nombre_categoria].copy()
        if category_df.empty: return None
        
        category_df['employment sector'] = category_df['employment sector'].str.replace('Ciencia y TecnologÃ­a; Science', 'Ciencia y Tecnología; Science', regex=False)
        top_sectors = category_df['employment sector'].value_counts().nlargest(3).index.tolist()
        analysis_df = category_df[category_df['employment sector'].isin(top_sectors)]
        if analysis_df.empty: return None

        perception_by_sector = analysis_df.groupby('employment sector')[['int_ans1', 'int_ans2', 'int_ans3']].mean()
        perception_by_sector.columns = ['Probabilidad', 'Impacto', 'Capacidad de Respuesta']
        
        plt.figure(figsize=(12, 7))
        ax = perception_by_sector.plot(kind='bar', figsize=(12, 7), width=0.8, edgecolor='black', zorder=3)
        plt.title('Percepciones Promedio por Sector - Bloque 1', fontsize=16, pad=20)
        plt.xlabel('Sector Laboral', fontsize=12)
        plt.ylabel('Valoración Promedio (Escala 1-10)', fontsize=12)
        plt.xticks(rotation=0, ha='center', fontsize=10)
        plt.grid(axis='y', linestyle='--', alpha=0.7, zorder=0)
        plt.legend(title='Variables de Percepción', fontsize=10)
        plt.tight_layout()
        
        output_filename = 'grafico_percepciones_Bloque_1.png'
        plt.savefig(output_filename)
        plt.close()
        return output_filename
    except Exception as e:
        print(f"Error al generar gráfico: {e}")
        return None

# --- 2. Carga de Datos y Ejecución de Análisis ---
nombre_archivo_excel = 'tablabase_plana3.xlsx'
try:
    df = pd.read_excel(nombre_archivo_excel)
    print(f"Archivo '{nombre_archivo_excel}' cargado exitosamente.")
    
    # --- Generar los artefactos necesarios para el informe ---
    
    # Generar el gráfico
    nombre_archivo_grafico = generar_grafico_percepciones(df, "Bloque 1: Incertidumbres críticas sobre la Agroindustria Alimentaria Argentina, en el contexto mundial (1 a 12) ")
    
    # Generar el scorecard enriquecido
    scorecard_enriquecido = crear_scorecard_enriquecido(df)
    
    # Generar el scorecard general
    scorecard_general = scorecard_enriquecido.groupby('email').agg(
        bloques_participados=('Bloque temático', 'count'),
        total_argumentos_globales=('argumentos_por_bloque', 'sum'),
        score_calidad_total_global=('score_calidad_total', 'sum'),
        score_calidad_promedio_global=('score_calidad_promedio', 'mean')
    ).round(2).reset_index()
    scorecard_general_sorted = scorecard_general.sort_values(by=['score_calidad_total_global', 'total_argumentos_globales'], ascending=False)
    
    # Generar la tabla de arquetipos
    mediana_cantidad = scorecard_general['total_argumentos_globales'].median()
    mediana_calidad = scorecard_general[scorecard_general['score_calidad_promedio_global'] > 0]['score_calidad_promedio_global'].median()
    def asignar_arquetipo(row):
        alta_calidad = row['score_calidad_promedio_global'] >= mediana_calidad and row['score_calidad_promedio_global'] > 0
        alta_cantidad = row['total_argumentos_globales'] >= mediana_cantidad and row['total_argumentos_globales'] > 0
        if row['total_argumentos_globales'] == 0: return 'Votante'
        if alta_calidad and alta_cantidad: return 'Experto Ancla'
        elif alta_calidad and not alta_cantidad: return 'Especialista Preciso'
        elif not alta_calidad and alta_cantidad: return 'Participante Amplio'
        else: return 'Votante'
    scorecard_general['arquetipo'] = scorecard_general.apply(asignar_arquetipo, axis=1)
    conteo_arquetipos = scorecard_general['arquetipo'].value_counts().reset_index()
    conteo_arquetipos.columns = ['Arquetipo', 'N° de Expertos']

    print("Todos los análisis necesarios para el informe han sido completados.\n")
    print("--- INICIO DEL INFORME MARKDOWN ---")

    # --- 3. CONSTRUCCIÓN DEL OUTPUT MARKDOWN ---

    # Carátula
    print("# **Análisis de Complejidad y Perfiles de Expertos en la Encuesta Delphi Agroindustrial 2012**")
    print("**Subtítulo:** Hallazgos Estratégicos y Metodologías de Scoring a partir de la Base de Datos Original.")
    print("**Autor:** Prueba _ TF UCES.")
    print("**Fecha:** 9 de Septiembre de 2025.")
    print("\n***\n")

    # Índice
    print("## **Índice**")
    print("1.  [Introducción](#1-introducción)")
    print("2.  [Metodologías de Análisis Desarrolladas](#2-metodologías-de-análisis-desarrolladas)")
    print("3.  [Resultados y Hallazgos Clave](#3-resultados-y-hallazgos-clave)")
    print("4.  [Conclusiones Estratégicas](#4-conclusiones-estratégicas)")
    print("\n***\n")

    # Introducción
    print("## **1. Introducción**")
    print("El presente informe tiene como objetivo presentar una serie de análisis y herramientas desarrolladas para profundizar en los resultados de la encuesta Delphi sobre el futuro de la agroindustria, realizada por el INTA en 2012. Yendo más allá de las conclusiones presentadas en los informes originales, este trabajo explora la complejidad subyacente de los datos, con un enfoque en la divergencia de las percepciones sectoriales y el análisis del comportamiento de los participantes. A través de la aplicación de técnicas de scoring y clasificación, se busca transformar la base de datos original en una herramienta de inteligencia estratégica que permita identificar patrones no evidentes, ponderar la calidad de las contribuciones y mapear el ecosistema de visiones que coexisten en el sector agroindustrial argentino.")
    print("\n***\n")

    # Metodologías
    print("## **2. Metodologías de Análisis Desarrolladas**")
    print("\n### **2.1 Scoring de Calidad de Argumentos**")
    print("Se estableció un marco para evaluar objetivamente la calidad de las justificaciones provistas por los expertos. El objetivo es asignar una puntuación a cada argumento (de 0 a 4) basada en un conjunto de características medibles que sirven como indicadores de su elaboración y fundamentación: Uso de Ejemplos, Uso de Datos, Lenguaje Causal y Visión a Futuro.")
    print("\n### **2.2 Análisis de Arquetipos de Participantes**")
    print("Se clasificó a cada experto en uno de cuatro perfiles de comportamiento (Experto Ancla, Especialista Preciso, Participante Amplio y Votante), cruzando el volumen total de sus argumentos contra la calidad promedio de los mismos.")
    print("\n***\n")

    # Resultados
    print("## **3. Resultados y Hallazgos Clave**")
    print("\n### **3.1 Visualización de la Divergencia Sectorial**")
    if nombre_archivo_grafico:
        print(f"A continuación, se presenta el gráfico de barras agrupadas para el 'Bloque 1'.")
        print(f"![Gráfico de Percepciones por Sector]({nombre_archivo_grafico})")
        print("\nEste gráfico evidencia las distintas 'personalidades' de los sectores. Se observa que el sector **Empresarial** percibe el mayor nivel de riesgo (Probabilidad e Impacto), mientras que el sector de **Ciencia y Tecnología** se muestra más cauto. Es notable el consenso respecto a la **Capacidad de Respuesta**.")
    else:
        print("No se pudo generar el gráfico para esta sección.")
    
    print("\n### **3.2 Ranking de Expertos por Calidad de Contribución**")
    print("La siguiente tabla muestra el ranking general de los 15 expertos más influyentes del estudio, ponderando tanto la cantidad como la calidad de sus aportes.")
    columnas_general_display = {'email': 'Email', 'bloques_participados': 'N° Bloques', 'total_argumentos_globales': 'N° Arg. Total', 'score_calidad_total_global': 'Score Total Global', 'score_calidad_promedio_global': 'Calidad Prom. Global'}
    display_general_df = scorecard_general_sorted.head(15).rename(columns=columnas_general_display)
    print(display_general_df.to_markdown(index=False))

    print("\n### **3.3 Distribución de Arquetipos en el Panel de Expertos**")
    print("La clasificación de los participantes según su comportamiento arroja la siguiente distribución:")
    print(conteo_arquetipos.to_markdown(index=False))
    print("\n***\n")

    # Conclusiones
    print("## **4. Conclusiones Estratégicas**")
    print("El análisis profundo de los datos permite afirmar la existencia de visiones sectoriales distintas y consistentes, así como la identidad de los expertos clave cuyas contribuciones cualitativas son de mayor peso. Este nivel de análisis permite una toma de decisiones más informada, ya que en lugar de buscar un único futuro promedio (una conclusión que no se puede afirmar), se pueden diseñar estrategias que consideren las diferentes lógicas y prioridades de cada grupo de actores. La principal conclusión, por lo tanto, no es una tendencia única, sino el entendimiento de que el futuro de la agroindustria se construirá sobre la tensión y, eventualmente, la convergencia de estas múltiples visiones.")

    print("\n\n--- FIN DEL INFORME MARKDOWN ---")

except FileNotFoundError:
    print(f"Error Crítico: No se encontró el archivo '{nombre_archivo_excel}'.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Archivo 'tablabase_plana3.xlsx' cargado exitosamente.
Todos los análisis necesarios para el informe han sido completados.

--- INICIO DEL INFORME MARKDOWN ---
# **Análisis de Complejidad y Perfiles de Expertos en la Encuesta Delphi Agroindustrial 2012**
**Subtítulo:** Hallazgos Estratégicos y Metodologías de Scoring a partir de la Base de Datos Original.
**Autor:** Prueba _ TF UCES.
**Fecha:** 9 de Septiembre de 2025.

***

## **Índice**
1.  [Introducción](#1-introducción)
2.  [Metodologías de Análisis Desarrolladas](#2-metodologías-de-análisis-desarrolladas)
3.  [Resultados y Hallazgos Clave](#3-resultados-y-hallazgos-clave)
4.  [Conclusiones Estratégicas](#4-conclusiones-estratégicas)

***

## **1. Introducción**
El presente informe tiene como objetivo presentar una serie de análisis y herramientas desarrolladas para profundizar en los resultados de la encuesta Delphi sobre el futuro de la agroindustria, realizada por el INTA en 2012. Yendo más allá de las conclusiones presentadas en

<Figure size 1200x700 with 0 Axes>